## Goal : find french and english comments in a given set of comments

In [91]:
# packages used afterwards
import pandas as pd
import numpy as np
from Language_Detection import *

In [ ]:
# Load the dataframe containing the comments to classify

#list = [['et oui mais bon je voudrais pas dire dass ich nicht hier bin, weil du','Hello there, what are you doing?','Bonjour, eh oui c\'est pas poli','Check out TalkToMeInKorean. They at least HAD something like that, don\'t know their current product offering.','als du es dir vorgestellt hast.','Don\'t have to time to try it now, but I love your website (and app) design. Nice popping colors.'],[-1,-1,-1,-1,-1,-1]]
#initial_df = pd.DataFrame(list, index=['body', 'body_lang']).T
initial_df = pd.read_pickle('Data/processed_comments')


## Language classification using the package langdetect

In [ ]:
# Perform langdetect classification (=python package)
langdetect_classification_df = langdetect_dataframe(initial_df,seed=4)
langdetect_classification_df.head(50)

In [ ]:
# Save it to a pickle file
langdetect_classification_df.to_pickle('Data/Classified/langdetect_classification')

In [ ]:
langdetect_classification_df[['body_lang']].value_counts()

## Remove undefined featuring urls and then perform a new language detection  

In [ ]:
# read file saved after having run the previous part 
df = pd.read_pickle('Data/Classified/langdetect_classification')

In [ ]:
# Remove all hyperlinks 
links = (df['body'].str.contains('http')) & (df['body_lang']=='U')
df1 = df[~links]

In [ ]:
# Run the package detectlang again, only on undefined comments, with another seed
df2 = langdetect_dataframe(df1[df1['body_lang']=='U'],seed = 0)

In [ ]:
# Save it to a pickle file
df1.to_pickle('Data/Classified/langdetect_classification_1')
df2.to_pickle('Data/Classified/langdetect_classification_2')

In [ ]:
# Read the file we saved before
df2= pd.read_pickle('Data/Classified/langdetect_classification_2')

In [ ]:
# Put the newly classified comments in the previous dataframe
df3 = df2.combine_first(df1)

In [ ]:
df3[['body_lang']].value_counts()

In [ ]:
df4_fr = df3[df3['body_lang']=='fr']
df4_en = df3[df3['body_lang']=='en']
df4_other = df3[df3['body_lang']=='N']
df4_undef = df3[df3['body_lang']=='U']

In [ ]:
N = df1[['author']].nunique().item()
df_sum_fr = df4_fr.groupby('author').number_of_words.agg('sum')
df_sum_en = df4_en.groupby('author').number_of_words.agg('sum')
print("There were ",N," authors with >5000 words")
print("And after this classification, ")
print((df_sum_fr >= 5000).sum(), "  for french")
print((df_sum_en >= 5000).sum(), " for english")

In [ ]:
# Save all frames to pickle files
df3.to_pickle('Data/Classified/langdetect_classification_3')
df4_fr.to_pickle('Data/Classified/french_comments')
df4_en.to_pickle('Data/Classified/english_comments')
df4_other.to_pickle('Data/Classified/other_language_comments')
df4_undef.to_pickle('Data/Classified/undefined_comments')


In [ ]:
df3       = pd.read_pickle('Data/Classified/langdetect_classification_3')
df4_fr    = pd.read_pickle('Data/Classified/french_comments')
df4_en    = pd.read_pickle('Data/Classified/english_comments')
df4_other = pd.read_pickle('Data/Classified/other_language_comments')
df4_undef = pd.read_pickle('Data/Classified/undefined_comments')

## Human classification ##

**Be careful with the order of the cells!**
````{verbatim}
---> Begining : load a file containing undefined comments
^  v
^  v
^  v      ....classify.....
^  v
^  v
<--- End : overwrite new language attribution (over the previous 'U') for each comment you classified
 
So, be sure you run the last cells to not lose what you've just classified ! 

In [99]:
# LOAD DATA TO CLASSIFY 
# df_non_splitted = the whole set of comments (fr,en,U and N)
# df_to_classify  = the comments whose language is still undefined 
df_non_splitted = pd.read_pickle('Data/Classified/test_1')


# This blocks will set body_lang = 'N' for any comments <= 2 words starting with /r, /u, or ![gif]
def fixer(body):
    if body.startswith('/u'):
        return "N"
    elif body.startswith('/r'):
        return "N"
    if body.startswith('![gif]'):
        return "N"  
    else:
        return 
def fixer_wrapper(dataframe):
    dataframe['body_lang_2'] = dataframe['body'].apply(fixer) 
    
fixer_wrapper(df_non_splitted)
df_non_splitted['body_lang'] = np.where((df_non_splitted['body_lang_2'] == "N") & (df_non_splitted['number_of_words'] < 2), df_non_splitted['body_lang_2'], df_non_splitted['body_lang'])

df_to_classify  = df_non_splitted[df_non_splitted['body_lang']=='U']

In [100]:
# Classify by hand until you're bored 
# Type : 0 for non english and non french,
#        1 for english
#        2 for french
# Type   e to exit the function 
df_partially_classified = human_class_df(df_to_classify)

In [104]:
# if wou wanrt to see what you classified (change the argument to see more lines)
df_partially_classified.head(50)

,author,body,created_utc,number_of_words,body_lang


In [108]:
# Put what has been classified into the non splitted dataframe
df_non_splitted_new = df_partially_classified.combine_first(df_non_splitted)

In [109]:
# save new data to pickle files
df_non_splitted_new.to_pickle('Data/Classified/test_1')